In [ ]:
# !pip install --quiet shap==0.39

# script to explore a few ways of interpreting a bert based classificaiton model


In [58]:
import transformers

from numpy.lib.histograms import _histogram_dispatcher
import torch
from transformers import AutoModelForSequenceClassification,AutoTokenizer

from torchnlp.encoders import Encoder
from torchnlp.encoders.text import stack_and_pad_tensors
from torchnlp.encoders.text.text_encoder import TextEncoder
# from tokenizer import Tokenizer
from classifier_one_label import Classifier

import argparse
import os
from datetime import datetime
from pathlib import Path
import bios
import numpy as np
import scipy as sp
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          TextClassificationPipeline)

import shap

In [83]:
def preprocess_trained_model(model_dir):
    '''
    This is a bit of a funny function to allow the easy use of transformers own pipeline classes.
    
    We have created custom bert based models which has different class structure to original bert base models/bert for sequence
    classification. Therefore we need to shift a few things around to enable transformers package to be happy...
    
    Could also re-write the pipeline function but this is easier for now.
    
    
    '''
    
    # first load the model
    trained_model = Classifier.load_from_checkpoint(model_dir)
    # load tokenizer directly from the trained model. Guarantees its the right one esentially.
    # tokenizer from trained model
    tokenizer = trained_model.tokenizer.tokenizer
    # assert that names are shared
    assert trained_model.transformer.name_or_path == tokenizer.name_or_path
    
    # now the funky business
    # load model from bert for sequence classification - then replace with trained state_dicts

    auto_bert = AutoModelForSequenceClassification.from_pretrained(trained_model.transformer.name_or_path)
    
    # now replace auto_bert.bert with trained_model.transformer
    auto_bert.bert = trained_model.transformer
    
    # same for classifier
    auto_bert.classifier = trained_model.classification_head
    
    # now set the config id2label
    class_labels_dict =  trained_model.data.label_encoder.token_to_index
    
    # set label2index
    auto_bert.config.label2id = class_labels_dict
    
    # we need to assign the label indices to their string counter part - need to switch key/values from class_labels_idx
    auto_bert.config.id2label = {class_labels_dict[k]:k for k in class_labels_dict}
    
    return auto_bert, tokenizer
    

In [84]:
# set trained model/checkpoint directory

model_dir = "../ckpts/icd9_triage/emilyalsentzer/Bio_ClinicalBERT/version_16-02-2022--10-26-36/best-checkpoint.ckpt"

# preprocess and return a bert model for sequence classification with our trained bert and classifier

model, tokenizer = preprocess_trained_model(model_dir)

2022-02-16 15:20:50.947 | INFO     | classifier_one_label:__init__:156 - Dataset probided was : icd9_triage
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-02-16 15:20:53.152 | WARNING

In [85]:
model.config.id2label

{0: 'AcuteMedicine',
 1: 'Cardiology',
 2: 'Gastroenterology',
 3: 'Neurology',
 4: 'Obstetrics',
 5: 'Oncology',
 6: 'Respiratory'}

In [86]:
model.config.label2id

{'AcuteMedicine': 0,
 'Cardiology': 1,
 'Gastroenterology': 2,
 'Neurology': 3,
 'Obstetrics': 4,
 'Oncology': 5,
 'Respiratory': 6}

In [30]:
def score_and_visualize(text):
    prediction = pipe([text])
    print(prediction[0])

    explainer = shap.Explainer(pipe)
    shap_values = explainer([text])

    shap.plots.text(shap_values)

In [ ]:
# pipe from transformer is weird and seems to expect just two classes


In [87]:
# using the transformers pipeline
pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, return_all_scores=True)



In [91]:
# explain the model's predictions on IMDB reviews
example_texts = ["patient has some crazy heart problems", "severe breathing problems", "no idea"]

explainer = shap.Explainer(pred)

In [92]:
# test on some examples
shap_values = explainer(example_texts)

Visualize the impact on all the output classes

In the plots below, when you hover your mouse over an output class you get the explanation for that output class. When you click an output class name then that class remains the focus of the explanation visualization until you click another class.

The base value is what the model outputs when the entire input text is masked, while
is the output of the model for the full original input. The SHAP values explain in an addive way how the impact of unmasking each word changes the model output from the base value (where the entire input is masked) to the final prediction value.

In [93]:
shap.plots.text(shap_values)

,,patient,has,some,crazy,heart,problems,
AcuteMedicine,0.0,0.043,-0.042,0.02,-0.06,0.022,0.014,0.0
Cardiology,-0.0,0.104,0.118,0.022,-0.067,-0.136,0.063,-0.0
Gastroenterology,-0.0,0.001,0.016,0.072,0.132,0.023,0.012,-0.0
Neurology,-0.0,0.081,0.147,-0.021,0.062,0.093,0.226,-0.0
Obstetrics,-0.0,0.022,0.014,-0.041,-0.077,0.445,0.028,-0.0
Oncology,0.0,0.033,-0.001,0.12,0.082,0.105,-0.036,0.0
Respiratory,0.0,-0.11,-0.102,-0.252,-0.277,-0.236,-0.123,0.0


,,severe,breathing,problems,
AcuteMedicine,0.0,0.15,0.07,-0.007,0.0
Cardiology,0.0,-0.049,-0.031,-0.025,0.0
Gastroenterology,-0.0,-0.044,0.127,0.001,-0.0
Neurology,-0.0,0.018,-0.09,0.324,-0.0
Obstetrics,0.0,0.116,-0.059,0.249,0.0
Oncology,0.0,0.167,0.068,-0.051,0.0
Respiratory,0.0,-0.218,-0.291,-0.132,0.0


,,no,idea,
AcuteMedicine,0.0,0.077,-0.042,0.0
Cardiology,-0.0,-0.247,-0.264,-0.0
Gastroenterology,-0.0,-0.253,-0.06,-0.0
Neurology,-0.0,-0.414,-0.176,-0.0
Obstetrics,-0.0,0.282,0.25,-0.0
Oncology,-0.0,-0.131,-0.109,-0.0
Respiratory,0.0,0.799,0.414,0.0


Visualize the impact on a single class

Since Explanation objects are sliceable we can slice out just a single output class to visualize the model output towards that class.

In [94]:
shap.plots.text(shap_values[:, :, "Cardiology"])

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# below is manual to use if transformers pipeline fucks up

In [54]:
# simpler predict function for bert model
# define a prediction function
model.cuda()

def sigmoid(_outputs):
    return 1.0 / (1.0 + np.exp(-_outputs))


def softmax(_outputs):
    maxes = np.max(_outputs, axis=-1, keepdims=True)
    shifted_exp = np.exp(_outputs - maxes)
    return shifted_exp / shifted_exp.sum(axis=-1, keepdims=True)

def predict_function(x):
    
    
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    print(f"outputs : {outputs.shape}")
    scores = softmax(outputs)
    print(f"scores : {scores.shape}")
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val


In [55]:
predict_function(["patient testing "])

outputs : (1, 7)
scores : (1, 7)


array([-3.0176475], dtype=float32)

In [61]:

# build an explainer using a token masker
explainer = shap.Explainer(predict_function, tokenizer)

# explain the model's predictions on IMDB reviews
example_texts = ["patient is healthy", "cheese smells good", "rabble rabble lols"]
shap_values = explainer(example_texts, fixed_context=1)

outputs : (1, 7)
scores : (1, 7)
outputs : (1, 7)
scores : (1, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (1, 7)
scores : (1, 7)
outputs : (1, 7)
scores : (1, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (1, 7)
scores : (1, 7)
outputs : (1, 7)
scores : (1, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (2, 7)
scores : (2, 7)
outputs : (4, 7)
scores : (4, 7)
outputs : (6, 7)
scores : (6, 7)
outputs : (2, 7)
scores : (2, 7)


In [63]:
# plot the first sentence's explanation
shap.plots.text(shap_values)

# below is just precursor to the clean functions above

In [67]:
# load model
model_dir = "../ckpts/icd9_triage/emilyalsentzer/Bio_ClinicalBERT/version_16-02-2022--10-26-36/best-checkpoint.ckpt"
model = Classifier.load_from_checkpoint(model_dir)

2022-02-16 15:07:46.388 | INFO     | classifier_one_label:__init__:156 - Dataset probided was : icd9_triage
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-02-16 15:07:48.780 | WARNING

In [18]:
model.transformer.name_or_path

'emilyalsentzer/Bio_ClinicalBERT'

In [74]:
labels_dict = model.data.label_encoder.token_to_index

In [75]:
{labels_dict[k]:k for k in labels_dict}

{0: 'AcuteMedicine',
 1: 'Cardiology',
 2: 'Gastroenterology',
 3: 'Neurology',
 4: 'Obstetrics',
 5: 'Oncology',
 6: 'Respiratory'}

In [ ]:
# we actually want to pull out the 

In [13]:
# tokenizer from trained model
tokenizer = model.tokenizer.tokenizer
tokenizer

PreTrainedTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [17]:
tokenizer.name_or_path

'emilyalsentzer/Bio_ClinicalBERT'

In [ ]:
tokenizer.decode([15000,5000,7899])

In [ ]:
# orginal tokenizer

org_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
org_tokenizer

In [ ]:
org_tokenizer.decode([15000,5000,7899])

In [4]:
# load model from bert for sequence classification - just wanna see what it looks like

auto_bert = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [6]:
# crude but now replace these with the trained model... different names of encoder etc but same shape and size etc
auto_bert.classifier = model.classification_head

In [7]:
auto_bert.bert = model.transformer

In [11]:
# model.transformer.state_dict()

In [9]:
# auto_bert.bert.state_dict()

OrderedDict([('embeddings.position_ids',
              tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
                       140, 1

In [15]:
# def classification_pipeline(model, tokenizer):
    
#     return_all_scores = False
    
#     """
#         Classify the text(s) given as inputs.
#         Args:
#             args (`str` or `List[str]`):
#                 One or several texts (or one list of prompts) to classify.
#             return_all_scores (`bool`, *optional*, defaults to `False`):
#                 Whether to return scores for all labels.
#             function_to_apply (`str`, *optional*, defaults to `"default"`):
#                 The function to apply to the model outputs in order to retrieve the scores. Accepts four different
#                 values:
#                 If this argument is not specified, then it will apply the following functions according to the number
#                 of labels:
#                 - If the model has a single label, will apply the sigmoid function on the output.
#                 - If the model has several labels, will apply the softmax function on the output.
#                 Possible values are:
#                 - `"sigmoid"`: Applies the sigmoid function on the output.
#                 - `"softmax"`: Applies the softmax function on the output.
#                 - `"none"`: Does not apply any function on the output.
#         Return:
#             A list or a list of list of `dict`: Each result comes as list of dictionaries with the following keys:
#             - **label** (`str`) -- The label predicted.
#             - **score** (`float`) -- The corresponding probability.
#             If `self.return_all_scores=True`, one such dictionary is returned per label.
#         """
#         result = super().__call__(*args, **kwargs)
#         if isinstance(args[0], str):
#             # This pipeline is odd, and return a list when single item is run
#             return [result]
#         else:
#             return result

#     def preprocess(self, inputs, **tokenizer_kwargs) -> Dict[str, GenericTensor]:
#         return_tensors = self.framework
#         return self.tokenizer(inputs, return_tensors=return_tensors, **tokenizer_kwargs)

#     def _forward(self, model_inputs):
#         return self.model(**model_inputs)

#     def postprocess(self, model_outputs, function_to_apply=None, return_all_scores=False):
#         # Default value before `set_parameters`
#         if function_to_apply is None:
#             if self.model.config.problem_type == "multi_label_classification" or self.model.config.num_labels == 1:
#                 function_to_apply = ClassificationFunction.SIGMOID
#             elif self.model.config.problem_type == "single_label_classification" or self.model.config.num_labels > 1:
#                 function_to_apply = ClassificationFunction.SOFTMAX
#             elif hasattr(self.model.config, "function_to_apply") and function_to_apply is None:
#                 function_to_apply = self.model.config.function_to_apply
#             else:
#                 function_to_apply = ClassificationFunction.NONE

#         outputs = model_outputs["logits"][0]
#         outputs = outputs.numpy()

#         if function_to_apply == ClassificationFunction.SIGMOID:
#             scores = sigmoid(outputs)
#         elif function_to_apply == ClassificationFunction.SOFTMAX:
#             scores = softmax(outputs)
#         elif function_to_apply == ClassificationFunction.NONE:
#             scores = outputs
#         else:
#             raise ValueError(f"Unrecognized `function_to_apply` argument: {function_to_apply}")

#         if return_all_scores:
#             return [{"label": self.model.config.id2label[i], "score": score.item()} for i, score in enumerate(scores)]
#         else:
#             return {"label": self.model.config.id2label[scores.argmax().item()], "score": scores.max().item()}
    